In [1]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 17075 (delta 29), reused 29 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17075/17075), 15.69 MiB | 33.83 MiB/s, done.
Resolving deltas: 100% (11723/11723), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 20.4 MB/s eta 0:00:0000:01


In [2]:
!cp /kaggle/input/cat-detection/custom.yaml /kaggle/working/

In [3]:
!cp -r /kaggle/input/cat-detection/training_data /kaggle/working/

In [4]:
!cat /kaggle/working/custom.yaml

train: /kaggle/working/training_data/images/train
val: /kaggle/working/training_data/images/val

nc: 1
names: ['Cat']

In [5]:
!python train.py --img 1280 --batch 32 --epochs 50 --data /kaggle/working/custom.yaml --weights yolov5s.pt --patience 6

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=/kaggle/working/custom.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=32, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplo

In [7]:
!python val.py --data /kaggle/working/custom.yaml --weights runs/train/exp/weights/best.pt --img 1280

val: data=/kaggle/working/custom.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=1280, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /kaggle/working/training_data/labels/val.cache... 14 images, 0 bac
val: WARNING ⚠️ /kaggle/working/training_data/images/val/1.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /kaggle/working/training_data/images/val/10.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /kaggle/working/training_data/images/val/11.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /kaggle/working/training_data/images/val/12.jpg: corrupt JPEG restored

In [29]:
#showing some issue with finding best.pt so moving it to working directory(yolov5)

!cp /kaggle/input/weights/best.pt /kaggle/working/yolov5/

_________________________________________________

**"Note: If you're using saved weights (best.pt) on a Windows system, you might encounter a 'PosixPath' issue. See the README for details."**

In [33]:
import torch
from pathlib import Path

# Load the original weights on the CPU(my machine dont have a gpu anyway)
checkpoint = torch.load("best.pt", map_location="cpu")

# Inspecting 'opt' and 'model.yaml' for PosixPath
print("opt:", checkpoint.get("opt"))
print("model.yaml:", getattr(checkpoint["model"], "yaml", None))

opt: {'weights': 'yolov5s.pt', 'cfg': '', 'data': '/kaggle/working/custom.yaml', 'hyp': {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}, 'epochs': 50, 'batch_size': 32, 'imgsz': 1280, 'rect': False, 'resume': False, 'nosave': False, 'noval': False, 'noautoanchor': False, 'noplots': False, 'evolve': None, 'evolve_population': PosixPath('data/hyps'), 'resume_evolve': None, 'bucket': '', 'cache': None, 'image_weights': False, 'device': '', 'multi_scale': False, 'single_cls': False, 'optimizer': 'SGD', 'sync_bn': False, 'workers': 8, 'project': 'runs/train', 'name': 'exp', 'exist_ok': Fa

In [34]:
# Load the checkpoint
checkpoint = torch.load("best.pt", map_location="cpu")

# Clean 'opt'
if "opt" in checkpoint:
    checkpoint["opt"] = {
        k: str(v) if isinstance(v, Path) else v  # Convert PosixPath to string
        for k, v in checkpoint["opt"].items()
    }

# Clean 'model.yaml' if necessary
if "model" in checkpoint and hasattr(checkpoint["model"], "yaml"):
    checkpoint["model"].yaml = {
        k: str(v) if isinstance(v, Path) else v
        for k, v in checkpoint["model"].yaml.items()
    }

# Save the fixed checkpoint
torch.save(checkpoint, "best_fixed.pt")
print("Fixed checkpoint saved as best_fixed.pt")

Fixed checkpoint saved as best_fixed.pt
